In [14]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('export_data.csv', header = 0)
dataset.set_index(dataset.columns[0], inplace=True)
print(dataset[-3:])
input_data = np.array(dataset[-480:])
mean = np.mean(input_data, axis=0)
std = np.std(input_data, axis=0) 
input_data = (input_data - mean)/std  
input_data = torch.Tensor(input_data).unsqueeze(0)
print(input_data.shape)

                  Temp (°C)  Dew Point Temp (°C)  Rel Hum (%)  \
Date/Time                                                       
2019-07-31 21:00       21.9                 18.1         79.0   
2019-07-31 22:00       22.1                 14.3         61.0   
2019-07-31 23:00       21.3                 12.8         58.0   

                  Wind Dir (10s deg)  Wind Spd (km/h)  Stn Press (kPa)  
Date/Time                                                               
2019-07-31 21:00                33.0             11.0           100.67  
2019-07-31 22:00                 2.0             11.0           100.69  
2019-07-31 23:00                 1.0             10.0           100.72  
torch.Size([1, 480, 6])


In [4]:
import torch
import torch.nn as nn

class SimpleLSTM(nn.Module):
    def __init__(self, hidden_size=100, num_layers=1):
        super().__init__()
        self.input_size = 6
        self.output_size = 1
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True) # 위의 설명 1. 을 참고하여 None을 채우세요.
        self.fc = nn.Linear(self.hidden_size, self.output_size) # 위의 설명 2. 를 참고하여 None을 채우세요.

    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size) # 위의 설명 3. 을 참고하여 None을 채우세요.
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size) # 위의 설명 3. 을 참고하여 None을 채우세요.
        return hidden, cell
    
    def forward(self, x):
        # hidden, cell state init
        h, c = self.init_hidden(x.size(0))
        h, c = h.to(x.device), c.to(x.device)
        out, (h, c) = self.lstm(x, (h, c))     
        final_output = self.fc(out[:, -1:, :])     
        final_output = torch.squeeze(final_output, dim = 1) # shape (100,1)

        return final_output

In [15]:
model = SimpleLSTM()    
pred = model(input_data)
pred = pred.item() * std[0] + mean[0] # de-normalization
print(pred) # this is prediction of 1st Aug, 23:00, Kingston, Ontario

# The actual temp
# http://climate.weather.gc.ca/climate_data/hourly_data_e.html?hlyRange=2008-07-15%7C2019-08-07&dlyRange=2008-07-15%7C2019-08-07&mlyRange=%7C&StationID=47267&Prov=ON&urlExtension=_e.html&searchType=stnProv&optLimit=specDate&StartYear=1840&EndYear=2019&selRowPerPage=25&Line=74&Month=8&Day=1&lstProvince=ON&timeframe=1&Year=2019


22.31875416888655
